In [1]:
import utils
import pandas as pd
from authorized_visits import monthly_aut_overall

In [2]:
visitors, events, devices, url_categories = utils.load_data(event_categories=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [10]:
"""
in the url_categories file given to us, the disease/disease_category fields are not standardized/clean
some fields contain multiple diseases, some contain weird characters, etc
this is an attempt to clean that and then calculate the %AUT more accurately per disease 
(we currently just use whatever is in the disease_category field)
"""

def strip_diseases(url_categories):    
    # strip out disease categories + diseases
    disease_categories = pd.DataFrame()
    for row in url_categories.fillna('').iterrows():
        d = {}
        r = row[1]
        d['url_category_id'] = r.url_category_id
        dc = r.disease_category
        dc = dc.replace('\n', '')
        for x in dc.split(','):
            dis = x.lower().strip()  
            if dis != '':
                d['disease_category'] = dis.title()
                disease_categories = disease_categories.append(d, ignore_index=True)

    diseases = pd.DataFrame()
    for row in url_categories.fillna('').iterrows():
        d = {}
        r = row[1]
        d['url_category_id'] = r.url_category_id
        dc = r.disease
        dc = dc.replace('\n', '')
        for x in dc.split(','):
            dis = x.lower().strip()
            if dis != '':
                d['disease'] = dis.title()
                diseases = diseases.append(d, ignore_index=True)
    return disease_categories, diseases

def disease_aut_monthly(disease_categories, diseases, visitors, events, url_categories):
    authorized_by_month = monthly_aut_overall(visitors, events)
    authorized_by_month = authorized_by_month[['url_category_id', 'month', 'authorized_visits', 'identified_visits', 'unk_sessions', 'total_visits']]
    # disease_categories
    d0 = pd.merge(authorized_by_month, disease_categories, on='url_category_id')
    d00 = d0.groupby(['disease_category']).url_category_id.nunique()
    d0 = d0.groupby(['disease_category', 'month'])\
            [['authorized_visits', 'identified_visits', 'unk_sessions', 'total_visits']].sum().reset_index()
    d0['pct_authorized'] = d0.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)
    d0['disease_category'] = d0.disease_category.str.title()

    # disease
    d1 = pd.merge(authorized_by_month, diseases, on='url_category_id')
    d1 = d1.groupby(['disease', 'month'])\
            [['authorized_visits', 'identified_visits', 'unk_sessions', 'total_visits']].sum().reset_index()
    d1['pct_authorized'] = d1.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)
    d1['disease'] = d1.disease.str.title()
    
    return d0, d1


In [4]:
disease_categories, diseases = strip_diseases(url_categories)

In [22]:
disease_categories.head()
# these join to url_categories on url_category_id

,disease_category,url_category_id
0,Surgery,0.0
1,Surgery,1.0
2,Surgery,2.0
3,Surgery,3.0
4,Surgery,4.0


In [11]:
d0, d1 = disease_aut_monthly(disease_categories, diseases, visitors, events, url_categories)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if __name__ == '__main__':
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if __name__ == '__main__':


In [13]:
d1

,disease,month,authorized_visits,identified_visits,unk_sessions,total_visits,pct_authorized
0,Advanced Prostate Cancer,3,4,4,89.0,93.0,0.043011
1,Advanced Prostate Cancer,4,6,7,191.0,198.0,0.030303
2,Arthritis,2,422,610,235603.0,236213.0,0.001787
3,Arthritis,3,367,509,291046.0,291555.0,0.001259
4,Arthritis,4,254,347,223031.0,223378.0,0.001137
5,Arthritis,5,297,386,253973.0,254359.0,0.001168
6,Arthritis,6,268,348,282496.0,282844.0,0.000948
7,Arthritis,7,338,430,354433.0,354862.0,0.000952
8,Birth Control,2,2132,3107,1794517.0,1797624.0,0.001186
9,Birth Control,3,1667,2369,2120623.0,2122992.0,0.000785
